# Scraping the Department of Corrections

In [ ]:
from urllib2 import Request, urlopen
from bs4 import BeautifulSoup
from re import compile, sub, match, findall
InmateLinks = []
for i in range(1,65):
    url = "http://www.criminaljustice.ny.gov/SomsSUBDirectory/search_index.jsp?offenderSubmit=true&LastName=&County="+str(i)+"&Zip=&Submit=Search"
    request = Request(url)
    request.add_header('From',"ram2233@columbia.edu")
    connection = urlopen(request)
    data = connection.read()
    page = BeautifulSoup(data)
    rows = page.findAll("td")
    for row in rows:
        link = row.findChild("a")
        if link and "http://www.criminaljustice.ny.gov"+str(link.get("href")) not in InmateLinks:
            InmateLinks.append("http://www.criminaljustice.ny.gov"+str(link.get("href")))
    print i

In [ ]:
InmateInfo = []
for inmates in InmateLinks:
    request = Request(inmates)
    request.add_header('From',"roger.miller@thenewyorkworld.com")
    connection = urlopen(request)
    data = connection.read()
    page = BeautifulSoup(data)
    rows = page.findAll("li")
    info = {"url":inmates}
    repeats = []
    for row in rows: 
        key = row.findChild("span","label")
        value = row.findChild("span","value")
        if key and value:
            if key.getText().strip() not in info:
                info[key.getText().strip()]=value.getText(separator=" ").encode("ascii","replace").replace("?"," ").replace("  ","").replace("\t","").strip()
            else:
                repeats.append(key.getText().strip())
                info[key.getText().strip()+str(repeats.count(key.getText().strip())+1)]=value.getText(separator=" ").encode("ascii","replace").replace("?"," ").replace("  ","").replace("\t","").strip()
    InmateInfo.append(info)

In [ ]:
len(InmateInfo)

In [ ]:
#started scrape at 10:46 am
from pandas import DataFrame

inmates = DataFrame(InmateInfo)

inmates

In [ ]:
from pandas import set_option
set_option("max_columns",500)
set_option("max_colwidth",500)
inmates["Color47"].value_counts()
inmates[inmates["Color47"]=="Orange"]

In [ ]:
inmates.to_csv("InmateInfo.csv")

# Analyzing the sex offender data

In [ ]:
from pandas import read_csv, set_option
set_option("max_columns",500)
set_option("max_colwidth",500)

o = read_csv(open("InmateInfo.csv"))

o["County"].value_counts()

In [ ]:
from re import match
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False
def zipcode(x):
    x = x.strip()
    if is_number(x[-5:]):
        return x[-5:]
    else:
        return "No Zip"

o["Zip"] = o["Address"].apply(zipcode)
o
        